# Assignment 5 #
### Due: Friday, November 17th to be submitted via Canvas by 11:59 pm ###
### Total points: **65** ###

#### **Group 62**
#### **UDIT DHAND (umd84)**
#### **SANYAM JAIN (sj33448)**

# Q1: Support Vector Machines (10 points)

In this question, we will explore support vector machines for the Spam Base dataset from the UCI repository.

In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [9]:
seed = 42

In [10]:
!pip install ucimlrepo

In [13]:
df = pd.read_csv('spambase.data', header=None)
y = df[57]
X = df.drop(columns=[57])


In [14]:
from ucimlrepo import fetch_ucirepo
# fetch dataset
spambase = fetch_ucirepo(id=94)

# data (as pandas dataframes)
X = spambase.data.features
y = spambase.data.targets
print("Abstract:", spambase.metadata['abstract'])
print("Number of instances:", spambase.metadata['num_instances'])
print("Number of features:", spambase.metadata['num_features'])
print(spambase.variables['name'])

Abstract: Classifying Email as Spam or Non-Spam
Number of instances: 4601
Number of features: 57
0                 word_freq_make
1              word_freq_address
2                  word_freq_all
3                   word_freq_3d
4                  word_freq_our
5                 word_freq_over
6               word_freq_remove
7             word_freq_internet
8                word_freq_order
9                 word_freq_mail
10             word_freq_receive
11                word_freq_will
12              word_freq_people
13              word_freq_report
14           word_freq_addresses
15                word_freq_free
16            word_freq_business
17               word_freq_email
18                 word_freq_you
19              word_freq_credit
20                word_freq_your
21                word_freq_font
22                 word_freq_000
23               word_freq_money
24                  word_freq_hp
25                 word_freq_hpl
26              word_freq_george
27          

In [15]:
X.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [32]:
seed = 42
y = y.to_numpy().squeeze()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=seed)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

a. (5 points) Implement the following function to train SVMs with a specified kernel type, hyper-parameter search space, and random state on the Spam Base dataset. Do hyper-parameter search over $C$ using [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), setting the number of folds to 5. After finding the best C, please use it to train the final model and return both the final model and the best C.

In [17]:
def search_best_svm(kernel, C_search_space, random_state):
    best_score = -np.inf
    for C in C_search_space:
        # Initialize an SVM classifier with the specified kernel type, C value, and random state
        ### START CODE ###
        svm_mod = SVC(kernel=kernel, C=C, random_state=random_state)
        ### END CODE ###


        # Evaluate accuracy scores using 5-fold cross-validation scores on training set
        ### START CODE ###
        scores = cross_val_score(svm_mod, X_train, y_train, cv=5 , scoring="accuracy")
        ### END CODE ###

        # Compute the average score and compare with the current best score to update the best C
        ### START CODE ###
        score = np.mean(scores)

        if score > best_score:
            best_score = score
            best_C = C

        ### END CODE ###
        print(f"C: {C} Avg Cross Val Score: {np.round(score, 4)}")

    print(f"Best C: {best_C}")

    # Initialize the model using the specified kernel type, best C, and random state;
    # and then fit the model using training set
    ### START CODE ###
    model = SVC(kernel=kernel, C=best_C, random_state=random_state)
    model.fit(X_train, y_train)
    ### END CODE ###
    return model, best_C

b. (3 points) Run the function you implemented above to train SVMs with the search space of $C$ being [$0.1, 1, 10, 100$], random state set to 42, with the following three popular kernels: (i) linear (ii) polynomial (iii) RBF (Gaussian). Evaluate your final models on the test set and report their accuracies.

In [19]:
search_space_c = [0.1,1,10,100]
kernel_type = ['linear', 'poly', 'rbf']
random_state = 42


model_accuracy_dict = {'linear': -np.inf, 'poly': -np.inf, 'rbf': -np.inf}

best_model = None
best_test_acc = -np.inf
best_kernel = None

for kernel in kernel_type:
    print(f"Kernel: {kernel}")
    model, best_C = search_best_svm(kernel, search_space_c, random_state)
    print(f"Training Accuracy: {np.round(model.score(X_train, y_train), 4)}")
    print(f"Test Accuracy: {np.round(model.score(X_test, y_test), 4)}")
    print("===========================================")


    if model.score(X_test, y_test) > best_test_acc:
        best_test_acc = model.score(X_test, y_test)
        best_model = model
        best_kernel = kernel
    
    model_accuracy_dict[kernel] = model.score(X_test, y_test)

print(f"Best Model: {best_model}")
print(f"Best Kernel: {best_kernel}")
print(f"Best C: {best_C}")
print(f"Best Test Accuracy: {best_test_acc}")
print("===========================================")
    

Kernel: linear
C: 0.1 Avg Cross Val Score: 0.9214
C: 1 Avg Cross Val Score: 0.9264
C: 10 Avg Cross Val Score: 0.9283
C: 100 Avg Cross Val Score: 0.9268
Best C: 10
Training Accuracy: 0.9355
Test Accuracy: 0.9283
Kernel: poly
C: 0.1 Avg Cross Val Score: 0.6895
C: 1 Avg Cross Val Score: 0.7594
C: 10 Avg Cross Val Score: 0.8428
C: 100 Avg Cross Val Score: 0.8957
Best C: 100
Training Accuracy: 0.9598
Test Accuracy: 0.9125
Kernel: rbf
C: 0.1 Avg Cross Val Score: 0.896
C: 1 Avg Cross Val Score: 0.9243
C: 10 Avg Cross Val Score: 0.9243
C: 100 Avg Cross Val Score: 0.9123
Best C: 10
Training Accuracy: 0.9681
Test Accuracy: 0.9354
Best Model: SVC(C=10, random_state=42)
Best Kernel: rbf
Best C: 10
Best Test Accuracy: 0.935361216730038


c. (2 points) Train a logistic regression model using the training set. Compare its performance with that of the SVMs trained above.

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Initialize a logistic regression classifier
log_reg = LogisticRegression(random_state=seed, max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_log = log_reg.predict(X_test)
acc_log = accuracy_score(y_test, y_pred_log)


print("Logistic Regression Model Test set Accuracy: ", round(acc_log, 4))

for key ,value in model_accuracy_dict.items():
    print(f"SVM {key} Test Set Accuracy: {round(value, 4)}")

Logistic Regression Model Test set Accuracy:  0.9223
SVM linear Test Set Accuracy: 0.9283
SVM poly Test Set Accuracy: 0.9125
SVM rbf Test Set Accuracy: 0.9354


We can see that the test set accuracy of logistic regression model is very similar to that of SVM models. However, SVM model with RBF kernel is slightly better than Logistic Regression model.

# Question 2 : Ensemble Methods for Classification (25 pts)

In this question, we will compare the performances of different ensemble methods for classification problems: [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html), [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) classifiers.

We will look at the [GiveMeSomeCredit](https://www.kaggle.com/c/GiveMeSomeCredit) dataset for this question. The dataset is extremely large so for this question we will only consider a subset which has been provided along with the notebook for this assignment. The dataset has already been split into train and test sets.

The task is to predict the probability that someone will experience financial distress in the next two years.

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
uploaded = files.upload()

Saving hw5_data.csv to hw5_data.csv


In [35]:
import pandas as pd

data = pd.read_csv('hw5_data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.571373,66,0,0.430620,9274.0,10,0,1,0,0.0
1,0,0.233999,56,0,0.257380,5656.0,12,0,0,0,0.0
2,0,0.299270,33,0,0.114575,4747.0,8,0,0,0,3.0
3,0,0.032165,41,0,0.308326,8490.0,8,0,1,0,0.0
4,0,0.050591,36,0,0.862627,3333.0,8,0,2,0,0.0


In [36]:
from sklearn.model_selection import train_test_split
y = data['SeriousDlqin2yrs']
X = data.drop(['SeriousDlqin2yrs'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 7)

print('train:',X_train.shape, y_train.shape)
print('test:',X_test.shape, y_test.shape)

train: (3750, 10) (3750,)
test: (1250, 10) (1250,)


In [37]:
import matplotlib.pyplot as plt
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from time import time
import xgboost
%matplotlib inline

In [38]:
columns_list = list(X.columns)

a. (2.5 pts) Fit a Decision Tree Classifier with random_state = 14 for this classification problem. Report the accuracy_score and roc_auc_score on the test set.

In [39]:
def fit_classifier(clf):
  # Fit the classifier on the training set
  ### START CODE ###
  clf.fit(X_train, y_train)
  ### END CODE ###
  return clf

In [40]:
def evaluate_classifier(clf, X_test, y_test):
  # Compute the accuracy_score, and roc_auc_score on the test set
  ### START CODE ###
  y_pred = clf.predict(X_test)
  y_pred_proba = clf.predict_proba(X_test)[:, 1]

  acc_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_pred_proba)
  ### END CODE ###
  print("Accuracy_score: {}, ROC_AUC_score: {}".format(acc_score, auc_score))

In [42]:
print("Decision Tree")
# Initialize your decision tree classifier
### START CODE ###
dt_clf = DecisionTreeClassifier(random_state=14)
### END CODE ###

dt_clf = fit_classifier(dt_clf)
evaluate_classifier(dt_clf, X_test, y_test)

Decision Tree
Accuracy_score: 0.888, ROC_AUC_score: 0.5854582176218127


b. (2.5 pts) Create a [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) of 25 classifiers (i.e, n_estimators=25) with random_state=14. Please use Decision Tree Classifier with random_state=14 as the base classifier. Report accuracy_score and roc_auc_score on the test data for this emsemble classifier.

In [43]:
print("Bagging of Decesion Trees")
# Initialize your bagging classifier
### START CODE ###
bag_clf = BaggingClassifier(DecisionTreeClassifier(random_state=14), n_estimators=25, random_state=14)
### END CODE ###

bag_clf = fit_classifier(bag_clf)
evaluate_classifier(bag_clf, X_test, y_test)

Bagging of Decesion Trees
Accuracy_score: 0.9272, ROC_AUC_score: 0.7866043928300934


c. (5 pts) In this question, you will fit a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) model on the training data for this classification task.

1. First, please find the best parameters (including *n_estimators*, *max_features* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the optimal parameters obtained by GridSearch.
2. Fit a model using the best parameters, and report the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score) on test data.

In [44]:
def grid_search_for_classifier(clf, param_grid, X_train, y_train):
  # Grid search
  grid_search = GridSearchCV(clf, param_grid=param_grid)

  # Conduct grid search using the training set (1 line of code only)
  ### START CODE ###
  grid_search.fit(X_train, y_train)
  ### END CODE ###
  print(grid_search.best_params_)

  # Set the best paramters for your clf (1 line of code only)
  ### START CODE ###
  clf.set_params(**grid_search.best_params_)
  ### END CODE ###
  return clf

In [45]:
def train_and_evaluate_classifier(clf, X_train, y_train, X_test, y_test):
  t0 = time()
  # Fit your classifier on the training set
  ### START CODE ###
  clf.fit(X_train, y_train)
  ### END CODE ###
  print("training time", round(time()-t0, 3), "s")

  t0 = time()
  y_pred = clf.predict(X_test)
  print("predict time", round(time()-t0, 3), "s")

  print("Confusion matrix: ")
  # Print the confusion matrix computed from the test set (1 line of code only)
  ### START CODE ###
  print(confusion_matrix(y_test, y_pred))
  ### END CODE ###


  ### START CODE ###
  y_pred_proba = clf.predict_proba(X_test)[:, 1]
  acc_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_pred_proba)
  ### END CODE ###

  print("Accuracy: {}, AUC_ROC: {}".format(acc_score, auc_score))
  return clf

In [46]:
param_grid = {"n_estimators": [1, 10, 50, 100],
              "max_features": [1, 5, 10, "auto"],
              "criterion": ['gini','entropy'],
              "random_state": [17]}

# Initialize your random forest classifier
### START CODE ###
rf_clf = RandomForestClassifier()
### END CODE ###
rf_clf = grid_search_for_classifier(rf_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(rf_clf, X_train, y_train, X_test, y_test)

{'criterion': 'entropy', 'max_features': 1, 'n_estimators': 100, 'random_state': 17}
training time 0.292 s
predict time 0.019 s
Confusion matrix: 
[[1160    5]
 [  81    4]]
Accuracy: 0.9312, AUC_ROC: 0.8378591264832114


RandomForestClassifier(criterion='entropy', max_features=1, random_state=17)

d. (10 pts) This time, let us use [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) for the same task. For AdaBoost and XGBoost, please respectively find the best parameters (including *n_estimators, learning_rate*); fit your model using the best parameters, and report the confusion matrix and roc_auc_score on test data.

In [47]:
param_grid = {"n_estimators": [10, 100],
          "learning_rate": [0.01, 0.1, 0.5],
          "random_state": [17]
          }

In [48]:
# Initialize your AdaBoost classifier
### START CODE ###
ab_clf = AdaBoostClassifier()
### END CODE ###
ab_clf = grid_search_for_classifier(ab_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(ab_clf, X_train, y_train, X_test, y_test)

{'learning_rate': 0.1, 'n_estimators': 100, 'random_state': 17}
training time 0.311 s
predict time 0.015 s
Confusion matrix: 
[[1153   12]
 [  72   13]]
Accuracy: 0.9328, AUC_ROC: 0.8390254986114618


AdaBoostClassifier(learning_rate=0.1, n_estimators=100, random_state=17)

In [49]:
# Initialize your XGBoost classifier
### START CODE ###
xgb_clf = xgboost.XGBClassifier()
### END CODE ###
xgb_clf = grid_search_for_classifier(xgb_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(xgb_clf, X_train, y_train, X_test, y_test)

{'learning_rate': 0.01, 'n_estimators': 100, 'random_state': 17}
training time 0.227 s
predict time 0.004 s
Confusion matrix: 
[[1149   16]
 [  78    7]]
Accuracy: 0.9248, AUC_ROC: 0.814915425397627


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=17, ...)

f. (5 pts) Compare the performance of decision tree from part a) with the ensemble methods. Briefly explain which of the three ensemble methods performed better and why?

**Compared to decision tree**: The ensemble methods (Bagging, RandomForest, AdaBoost) were all better than just using one decision tree. They all got higher scores for accuracy and ROC_AUC.

**Reason**: These methods work well because they use more than one model to make a guess. This usually makes them more right and less likely to make mistakes than just one model. In other words, they reduce both bias and variance.

**Comparing the Ensembe Methods**:

- AdaBoost performs best: It's a little more right than the others and has better ROC_AUC scores. It works by paying more attention to where it went wrong before and trying to fix those mistakes.

- RandomForest is almost as good: It's very close to AdaBoost and does a good job too. It makes a lot of decision trees and picks the answer that most trees agree on. It's really good for different kinds of data but might not be as sharp as AdaBoost.

- Bagging is better than DT but not the best: It's much better than just one tree because it makes less mistakes. But it doesn't do as good a job as AdaBoost or RandomForest because it doesn't focus on the parts where it's wrong.

**Conclusion**: AdaBoost did the best job with this data because it really focuses on what it got wrong before and keeps getting better each time.

# Q3: CatBoost (10 points)

In this question you will learn about a boosting algorithm known as **CatBoost**. Please go through the two videos specified below to get a better understanding of the CatBoost algorithm and answer the questions that follow.

[Part-1](https://www.youtube.com/watch?v=KXOTSkPL2X4&ab_channel=StatQuestwithJoshStarmer)
[Part - 2](https://www.youtube.com/watch?v=3Bg2XRFOTzg&t=242s&ab_channel=StatQuestwithJoshStarmer)



a. **(5 points)** Briefly explain Ordered Target Encoding. What challenge does it try to address?

b. **(5 points)** Briefly describe the main advantages and disadvantages of CatBoost as compared to XGBoost.

#### Answer a.

**Ordered Target Encoding** turns categories into numbers for machine learning. It's good for when you have colors or other categories that don't have numbers. 

Instead of making lots of new columns for each category, or just giving each one a random number, Ordered Target Encoding gives each category a number that means something important. Like if you're looking at house prices, it would give each neighborhood a number that's the average price of houses there.

**Using Ordered Target Encoding:**

Ordered Target Encoding deals with some big problems when turning categories into numbers for machine learning:

- It keeps all the important stuff and works well when there are lots of categories (high cardinality). Other ways like One-Hot Encoding can make too many columns if there are many categories, which can slow things down and make models too specific (overfit). Label Encoding can give categories random numbers that don't really match up with what you're trying to predict. But Ordered Target Encoding keeps the categories simple and still shows how they're related to what you're trying to predict. It uses an important number from what you're trying to predict (like the average price) for each category.

- It shows how categories affect what you're trying to predict. It uses the target thing itself to make the numbers, so it really captures how each category impacts the outcome.

So, Ordered Target Encoding changes categories into numbers in a way that keeps their link to what you're trying to predict. It helps keep things simple, stops the model from being too specific, and keeps the important links between the data.

#### Answer b.

Comparing CatBoost and XGBoost:

1. **Categorical Data:**
   - **CatBoost:** Great at working with categories without needing extra steps.
   - **XGBoost:** You have to get categories ready first, which takes extra work.<br><br>

2. **Speed and Performance:**
   - **CatBoost:** Works well with categories but might be slower with just numbers.
   - **XGBoost:** Usually trains faster, especially with numbers.<br><br>

3. **Overfitting:**
   - **CatBoost:** Better at not fitting too much, good for small data.
   - **XGBoost:** Can fit too much if you're not careful with the settings.<br><br>

4. **Help and Learning Resources:**
   - **CatBoost:** Has some support but not as much as XGBoost.
   - **XGBoost:** Lots of help and guides out there.<br><br>

5. **Parameter Tuning:**
   - **CatBoost:** Easier to start with but not as many tweaks you can make.
   - **XGBoost:** Lots of tweaks possible, but you need to know what you're doing.<br><br>

In short, use CatBoost if you have categories in your data. Pick XGBoost for number data or when you want to make a lot of specific adjustments to your model.

# Q4: Convolutional Neural Network (20 points)
In this question, we will continue our exercise on the SVHN classification task from the previous homework, but this time we will be using Convolutional Neural Networks.

In [50]:
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [51]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [52]:
transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614))
        ])

train_dataset = torchvision.datasets.SVHN(root='.', split='train', transform=transform, download=True)
test_dataset = torchvision.datasets.SVHN(root='.', split='test', transform=transform, download=True)

100%|██████████| 182040794/182040794 [00:23<00:00, 7879576.26it/s] 


100%|██████████| 64275384/64275384 [00:20<00:00, 3062800.40it/s]


In [53]:
train_num = int(len(train_dataset) * 0.8)
val_num = len(train_dataset) - train_num
# Randomly split the training dataset into training dataset and validation dataset
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_num, val_num])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

a. (10 points) Build a convolutional neural network with the following sequential configuration. If not specified, please use the default setting of torch.nn.Conv2d. The output of the convolution layers will be fed into a fully-connected MLP. Then train the model with Adam optimizer (lr=1e-3) for 10 epochs. You should be able to achieve test accuracy of over 85%.



> Layer 1
*   2d convolution (# input channel=3, # output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 1
*   2d max pooling (kernel size=2)

> Layer 2
*   2d convolution (# output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 2
*   2d max pooling (kernel size=2)

> Layer 3
*   2d convolution (# output channel=32, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 3
*   2d max pooling (kernel size=2)

References:

*   https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html






In [54]:
class CNN(nn.Module):
    def __init__(self, pool=True):
        super(CNN, self).__init__()
        self.pool = pool

        # Create convolutional layers
        ### START CODE ###
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        
        self.pool1 =  nn.MaxPool2d(kernel_size=2) if self.pool else None

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.pool2 = nn.MaxPool2d(kernel_size=2) if self.pool else None

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        self.pool3 = nn.MaxPool2d(kernel_size=2) if self.pool else None
        ### END CODE ###

        # Create fully connected layers (nn.Linear)
        if self.pool:
            self.mlp1 = nn.Linear(32*4*4, 50)
        else:
            self.mlp1 = nn.Linear(32*32*32, 50)

        self.mlp2 = nn.Linear(50, 50)
        self.mlp3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.layer1(x)
        if self.pool:
            x = self.pool1(x)

        x = self.layer2(x)
        if self.pool:
            x = self.pool2(x)

        x = self.layer3(x)
        if self.pool:
            x = self.pool3(x)
        x = x.reshape(x.shape[0], -1)

        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = self.mlp3(x)

        return x

In [55]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Use the optimizer to perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    return avg_loss

@torch.no_grad()
def eval(model, loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Get model's prediction
        pred = torch.argmax(out, dim=1)

        # Count number of correct predictions
        correct = accuracy_score(target, pred, normalize=False)

        total_correct += correct
        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    acc = total_correct / total_num
    return avg_loss, acc

In [56]:
model1 = CNN(pool=True)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model1, train_loader, optimizer)
    val_loss, val_acc = eval(model1, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model1 = model1
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

100%|██████████| 58/58 [00:03<00:00, 15.51it/s]


Epoch: 1 Train Loss: 1.2216163216307652 Val Loss: 0.6390311451124878 Val Acc: 0.8097188097188097


100%|██████████| 58/58 [00:04<00:00, 14.34it/s]


Epoch: 2 Train Loss: 0.5604904653065726 Val Loss: 0.49842555526010396 Val Acc: 0.855036855036855


100%|██████████| 58/58 [00:04<00:00, 13.29it/s]


...

Epoch: 8 Train Loss: 0.3197165992828311 Val Loss: 0.3769732241971617 Val Acc: 0.8881381381381381


100%|██████████| 58/58 [00:03<00:00, 16.26it/s]


Epoch: 9 Train Loss: 0.3017301185099951 Val Loss: 0.3783684137241456 Val Acc: 0.8873191373191374


100%|██████████| 58/58 [00:03<00:00, 16.40it/s]

Epoch: 10 Train Loss: 0.2924189313306671 Val Loss: 0.37169599677089 Val Acc: 0.8915506415506416


In [57]:
_, test_acc = eval(best_model1, test_loader)
print(f"Test accuracy: {np.round(test_acc, 3)}")

100%|██████████| 102/102 [00:06<00:00, 16.10it/s]

Test accuracy: 0.89


b. (5 points) Use torch-summary to print a summary of the model. The number of parameters should be less than the one of the MLP we trained in the previous homework. Why does it have less number of parameters but have higher accuracy?

Reference
*   https://pypi.org/project/torch-summary/


In [58]:
!pip install torch-summary

In [61]:
from torchsummary import summary

summary(model1, input_size=(3, 32, 32))

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       448
|    └─BatchNorm2d: 2-2                  32
|    └─ReLU: 2-3                         --
├─MaxPool2d: 1-2                         --
├─Sequential: 1-3                        --
|    └─Conv2d: 2-4                       2,320
|    └─BatchNorm2d: 2-5                  32
|    └─ReLU: 2-6                         --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Conv2d: 2-7                       4,640
|    └─BatchNorm2d: 2-8                  64
|    └─ReLU: 2-9                         --
├─MaxPool2d: 1-6                         --
├─Linear: 1-7                            25,650
├─Linear: 1-8                            2,550
├─Linear: 1-9                            510
Total params: 36,246
Trainable params: 36,246
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       448
|    └─BatchNorm2d: 2-2                  32
|    └─ReLU: 2-3                         --
├─MaxPool2d: 1-2                         --
├─Sequential: 1-3                        --
|    └─Conv2d: 2-4                       2,320
|    └─BatchNorm2d: 2-5                  32
|    └─ReLU: 2-6                         --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─Conv2d: 2-7                       4,640
|    └─BatchNorm2d: 2-8                  64
|    └─ReLU: 2-9                         --
├─MaxPool2d: 1-6                         --
├─Linear: 1-7                            25,650
├─Linear: 1-8                            2,550
├─Linear: 1-9                            510
Total params: 36,246
Trainable params: 36,246
Non-trainable params: 0

CNNs have fewer parameters and get better results than MLPs for a couple of reasons:

- **They use parameters better**: CNNs share weights and only connect some neurons together, which means they need fewer parameters. This helps them learn without overfitting.

- **They're good at picking out important parts of images**: CNNs can automatically find important patterns in images, like edges and textures. This is something MLPs can't do, and it's why CNNs are really good for working with images.

- **They can handle new data well**: CNNs are built in a way that helps them work well with data they haven't seen before. That's key for doing well on tasks like telling what's in an image.

Conclusion: CNNs need fewer parameters because of how they share weights and connect neurons, and they're more accurate at image tasks because they're really good at learning from the patterns in the data.

c. (5 points) Train another CNN with the pool option set to False. What are the differences in terms of accuracy or computation caused by disabling max pooling? What are the effects of pooling operations in CNNs? (This might take some time. Watch a TV show while you're waiting for the results..)

In [59]:
model2 = CNN(pool=False)
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model2, train_loader, optimizer)
    val_loss, val_acc = eval(model2, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model2 = model2
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

100%|██████████| 58/58 [00:07<00:00,  8.28it/s]


Epoch: 1 Train Loss: 1.1319699598674775 Val Loss: 0.5539264180297054 Val Acc: 0.834971334971335


100%|██████████| 58/58 [00:06<00:00,  8.31it/s]


Epoch: 2 Train Loss: 0.5003900743732984 Val Loss: 0.5062575723071779 Val Acc: 0.8523068523068523


100%|██████████| 58/58 [00:08<00:00,  6.95it/s]


...

Epoch: 8 Train Loss: 0.23433508727750052 Val Loss: 0.4333886660629369 Val Acc: 0.8792656292656292


100%|██████████| 58/58 [00:07<00:00,  8.24it/s]


Epoch: 9 Train Loss: 0.2091996848522198 Val Loss: 0.4426247705096532 Val Acc: 0.8793338793338793


100%|██████████| 58/58 [00:09<00:00,  5.95it/s]

Epoch: 10 Train Loss: 0.19314123682805515 Val Loss: 0.45493219819285 Val Acc: 0.8788561288561288


In [60]:
_, test_acc = eval(best_model2, test_loader)
print(test_acc)

100%|██████████| 102/102 [00:15<00:00,  6.43it/s]

0.8651659496004918


Pooling in CNNs affects how they work in a few key ways:

**Size of the Feature Maps:**
- **With Pooling:** It makes the feature maps smaller, so there are fewer things to calculate and fewer parameters. This helps stop the model from fitting too closely to the training data.
- **Without Pooling:** The feature maps stay bigger, leading to more calculations and more parameters.

**Learning Features:**
- **With Pooling:** Pooling helps the model learn more general features. It sort of summarizes what's in different parts of the input, making the model less affected by small shifts in where features are.
- **Without Pooling:** The model keeps more detailed information about where things are in the input. This can be good or bad – it's more detailed, but it might make the model too sensitive to small changes.

**Generalizing and Overfitting:**
- **With Pooling:** Generally, pooling makes the model better at working with new, unseen data. It becomes less bothered by small changes and noise in the input.
- **Without Pooling:** The model might learn the training data too well (overfit) because it has more parameters and keeps more detailed information.

**Accuracy and Computational Costs:**
- Disabling max pooling impacts both how accurate the model is and how much computing power it needs.
- A model with max pooling got a higher accuracy (89.15%) than one without it (87.8%). This shows that pooling helps the model generalize better and pick out stronger features, which is key for doing well with new data.
- Max pooling also means less computation – fewer parameters and less complexity. So, the model with pooling was faster and needed less memory. But, you do lose some detailed information about where things are, which might matter for some tasks.

In summary, using pooling in a CNN is a balance. It can make the model faster, less memory-hungry, and more accurate for general tasks, but you might lose some detailed spatial information that's important for certain kinds of jobs.